In [1]:
using NamedTuples
using DataStructures

In [2]:
Sym = Symbol
SymExpr = Expr

symExpr(x) = x
mathy = Union{Sym,SymExpr,Number};

function Base.:+(x::mathy, y::mathy)
    if x == y
        symExpr(:(2*$x))
    elseif x == -y
        0
    elseif x == 0
        y
    elseif y == 0
        x
    else
        symExpr(:($x+$y))
    end
end

Base.:+(x::mathy) = x

function Base.:-(x::mathy, y::mathy)
    if x == y
        0
    elseif x == -y
        symExpr(:(2$x))
    elseif x == 0
        -y
    elseif y == 0
        x
    else
        symExpr(:($x-$y))
    end
end

function Base.:-(x::Sym)
    symExpr(:(-$x))
end

isUnaryOperation(ex::SymExpr) = length(ex.args) == 2
car(x::SymExpr) = x.args[1]

function Base.:-(x::SymExpr)
    if (car(x) == :-) && (x |> isUnaryOperation)
        symExpr(x.args[2])
    else
        symExpr(:(-$x))
    end
end

function Base.:*(x::mathy,y::mathy)
    if x == y
        symExpr(:($x^2))
    elseif x == -y
        symExpr(:(-$x^2))
    elseif x == 1
        y
    elseif y == 1
        x
    elseif (x == 0) || (y == 0)
        0
    else
        symExpr(:($x*$y))
    end
end

function Base.:/(x::mathy, y::mathy)
    if x == y
        1
    elseif x == -y
        -1
    elseif y == 1
        x
    else
        symExpr(:($x/$y))
    end
end

function Base.:^(x::mathy, y::mathy)
    if y == 0
        1
    elseify == 1
        x
    else
        symExpr(:($x^$y))
    end
end

Base.:^(x::mathy, y::Int) = y == 1 ? x : y == 0 ? 1: symExpr(:($x^$y))

Base.log(x::mathy) = symExpr(:(log($x)))

In [3]:
function hasDuplicates(arr)
    for i in 1:(length(arr)-1)
        for j in (i+1):length(arr)
            if arr[i] == arr[j]
                return true
            end
        end
    end
    false
end

hasDuplicates (generic function with 1 method)

In [4]:
tagCount = 0

type DTag
    tag::Array
end

DTag(x) = DTag([x])

DTag(x...) = DTag([i for i in x] |> sort)

Base.length(t::DTag) = length(t.tag)
Base.:(==)(x::DTag,y::DTag) = x.tag == y.tag
Base.getindex(t::DTag, i::Int) = (t.tag)[i]

function Base.isless(t1::DTag,t2::DTag)
    if length(t1) == length(t2)
        for i in 1:length(t1)
            if t1[i] >= t2[i]
                return false
            end
        end
        t1 == t2 ? (return false ) : (return true)
    elseif length(t1) < length(t2)
        true
    else
        false
    end
end  

In [5]:
type Differential
    terms::SortedDict
end

function printEpsilons(t::DTag)
    str = ""
    for i in t.tag 
        str = str*"ϵ$i" 
    end
    str
end

function Base.show(io::IO, diff::Differential)
    str = ""
    for (tag, term) in diff.terms
        if tag == DTag()
            str = str*"$(term)$(printEpsilons(tag))"
        elseif str == ""
            str = str*"($(term))$(printEpsilons(tag))"
        else
            str = str*" + ($(term))$(printEpsilons(tag))"
        end
    end
    print(io, str)
end

function Differential(iterable)
    Differential(try delete!(SortedDict(iterable),DTag(-1)) catch SortedDict(iterable) end)
end

function Differential(keys::Union{Array,Tuple}, values::Union{Array,Tuple})
    Differential(try delete!(SortedDict(zip(keys,values)), DTag(-1)) catch SortedDict(zip(keys,values)) end)
end

Base.length(t::Differential) = length(t.terms)
Base.:(==)(x::Differential,y::Differential) = x.terms == y.terms
Base.getindex(t::Differential, i::DTag) = (t.terms)[i]
getTagList(Dx::Differential) = [key for (key, _) in Dx.terms]

Base.getindex(Dx::Differential, i::DTag) = Dx.terms[i]

function Base.getindex(Dx::Differential, i::Int)
    key = getTagList(Dx)[i]
    key == DTag() ? Dx.terms[key] : Differential(key => Dx.terms[key])
end

function Base.getindex(Dx::Differential, i::UnitRange)
    keys = getTagList(Dx)[i]
    if length(i) == 1 
        out = (Dx.terms)[keys...]
    else
        Differential(k => Dx.terms[k] for k in keys)
    end
end

Base.endof(Dx::Differential) = length(Dx |> getTagList)

lastTag(Dx::Differential) = getTagList(Dx)[end]

function tagUnion(x::Differential, y::Differential)
    vcat(x |> getTagList, y |> getTagList) |> sort |> fastuniq
end


mathyDiff = Union{mathy, Differential};

In [6]:
function fastuniq(v)
  v1 = Vector{eltype(v)}()
  if length(v)>0
    laste = v[1]
    push!(v1,laste)
    for e in v
      if e != laste
        laste = e
        push!(v1,laste)
      end
    end
  end
  return v1
end



fastuniq (generic function with 1 method)

In [7]:
function Base.:+(x::Differential, y::mathy)
    x + Differential([DTag()], [y])
end

function Base.:+(x::mathy, y::Differential)
    Differential([DTag()], [x]) + y
end


function Base.:+(x::Differential, y::Differential)
    tags = tagUnion(x,y)
    values = [((try x[tag] catch 0 end) + (try y[tag] catch 0 end)) for tag in tags]
    Differential(tags, values)
end

Dx = Differential(SortedDict(DTag() => :x^2, DTag(1) => :2*:x, DTag(2) => 2*:x, DTag(1,2) => 2))

ykeys = [DTag(),DTag(1)]
yvals = [:x, 1]

Dy = Differential(ykeys, yvals)

Dx + Dy

x ^ 2 + x + (2x + 1)ϵ1 + (2x)ϵ2 + (2)ϵ1ϵ2

In [8]:
function Base.:*(x::Differential, y::mathy)
    Differential(tag => value * y for (tag, value) in x.terms)
end

function Base.:*(y::mathy, x::Differential)
    Differential(tag => y * value for (tag, value) in x.terms)
end

function Base.:*(x::Differential, y::Differential)
    out = Differential([],[0])
    for (k1,v1) in x.terms
        for (k2,v2) in y.terms
            out += Differential([k1*k2], [v1*v2])
        end
    end
    out
end

function Base.:*(t1::DTag,t2::DTag)
    if vcat(t1.tag,t2.tag) |> hasDuplicates
        DTag(-1)
    else
        DTag(vcat(t1.tag,t2.tag) |> sort)
    end
end

Differential([DTag(1)*DTag(1)],[1]).terms

DataStructures.SortedDict{DTag,Int64,Base.Order.ForwardOrdering} with 0 entries

In [9]:
function unaryOp(f::Function, Df::Function)
    function (Dx::Differential) 
        f(Dx[1:end-1]) + Df(Dx[1:end-1])*Dx[end]
    end
end 

function binaryOp(f::Function, Dfx::Function, Dfy::Function)
    function (Dx::Differential, Dy::Differential) 
        f(Dx[1:end-1],Dy[1:end-1]) + Dfx(Dx[1:end-1], Dy[1:end-1])*Dx[end] + Dfy(Dx[1:end-1], Dy[1:end-1])*Dy[end]
    end
end

Base.:-(Dx::Differential, y::mathy) = unaryOp(x -> x - y, x -> 1)(Dx)
Base.:-(x::mathy, Dy::Differential) = unaryOp(y -> x - y, y -> -1)(Dy)
Base.:-(Dx::Differential, Dy::Differential) = binaryOp((x,y) -> x - y,
                                                       (x,y) -> 1,
                                                       (x,y) -> -1)(Dx,Dy)

Base.:/(Dx::Differential,y::mathy) = unaryOp(Dx -> Dx/y, Dx -> 1/y)(Dx)
Base.:/(x::mathy,Dy::Differential) = unaryOp(Dy -> x/Dy, Dy -> -1/(Dy)^2)(Dx)
Base.:/(Dx::Differential,Dy::Differential) = binaryOp((x,y) -> x/y, 
                                                      (x,y) -> 1/y,
                                                      (x,y) -> -x/y^2)(Dx,Dy)

Base.:^(Dx::Differential,y::mathy) = unaryOp(Dx -> Dx^y, Dx -> y*Dx^(y-1))(Dx)
Base.:^(Dx::Differential,y::Int) = unaryOp(Dx -> Dx^y, Dx -> y*Dx^(y-1))(Dx)
Base.:^(x::mathy,Dy::Differential) = unaryOp(Dy -> x^Dy, Dy -> log(x)*x^Dy)(Dy)

Base.log(Dx::Differential) = unaryOp(log, x -> 1/x)(Dx)

ϵ = Differential([DTag(1)],[1])

Dx = Dy

log(log(:x + ϵ))



log(log(x)) + ((1 / log(x)) * (1 / x))ϵ1

In [30]:
function D(f::Function)
    ϵ = makeDiff()
    x -> extractDiff(f(x + ϵ), ϵ)
end

function makeDiff()
    global tagCount += 1
    Differential([DTag(tagCount)],[1])
end

function extractDiff(Dx::Differential, ϵ::Differential)
    tag = (ϵ |> getTagList)[1]
    out = Differential(tagRemove(t, tag) => v for (t, v) in Dx.terms)
    if getTagList(out) == [DTag()]
        out[1]
    else
        out
    end
end
extractDiff(ex::Expr, ϵ::Differential) = 0
            

extractDiff (generic function with 2 methods)

In [29]:
Base.setdiff(t1::DTag, t2::DTag) = DTag(setdiff(t1.tag, t2.tag) |> sort)
Base.intersect(t1::DTag, t2::DTag) = DTag(intersect(t1.tag, t2.tag) |> sort)

tagRemove(t1::DTag, t2::DTag) = intersect(t1,t2) != DTag() ? setdiff(t1,t2) : DTag(-1)
t1 = DTag([1,2])
t2 = DTag([1])

tagRemove(t1,t2)

DTag([2])

DTag([2])

In [40]:
D(D(D(D(D(D(x -> x^5 + x))))))(:x)

0